In [1]:
import os
import pyvtk
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from netCDF4 import Dataset
import shutil
import glob

In [2]:
# The data structure in element-wise output is too complicated for xarray.open_mfdataset.
# Here we open the files as individual datasets and concatenate them on the variable level.
# This code is compatible with parallel netcdf build (single file output)

# load_wave_data=True:  read wave data and return numpy.ndarray
# load_wave_data=False: do not read wave data and return xarray.DataArray (use False if data is big)

def read_element_output(data_dir, load_wave_data=True):
    ################ open files ################
    # filenames
    nc_fnames = [f for f in os.listdir(data_dir) if 'axisem3d_synthetics.nc' in f]
    # print('files to open: ', nc_fnames)

    # open files
    nc_files = []
    for nc_fname in nc_fnames:
        nc_files.append(xr.open_dataset(data_dir + '/' + nc_fname))
    
    ################ variables that are the same in the datasets ################
    # read Na grid (all azimuthal dimensions)
    na_grid = nc_files[0].data_vars['list_na_grid'].values.astype(int)

    # read time
    data_time = nc_files[0].data_vars['data_time'].values
    
    
    ################ variables to be concatenated over the datasets ################
    # define empty lists of xarray.DataArray objects
    xda_list_element_na = []
    xda_list_element_coords = []
    dict_xda_list_element = {}
    dict_xda_data_wave = {}
    for nag in na_grid:
        dict_xda_list_element[nag] = []
        dict_xda_data_wave[nag] = []
    
    # loop over nc files
    for nc_file in nc_files:
        # append DataArrays
        xda_list_element_na.append(nc_file.data_vars['list_element_na'])
        xda_list_element_coords.append(nc_file.data_vars['list_element_coords'])
        for nag in na_grid:
            dict_xda_list_element[nag].append(nc_file.data_vars['list_element__NaG=%d' % nag])
            dict_xda_data_wave[nag].append(nc_file.data_vars['data_wave__NaG=%d' % nag])
            
    # concat xarray.DataArray
    xda_list_element_na = xr.concat(xda_list_element_na, dim='dim_element')
    xda_list_element_coords = xr.concat(xda_list_element_coords, dim='dim_element')
    for nag in na_grid:
        dict_xda_list_element[nag] = xr.concat(dict_xda_list_element[nag], dim='dim_element__NaG=%d' % nag)
        dict_xda_data_wave[nag] = xr.concat(dict_xda_data_wave[nag], dim='dim_element__NaG=%d' % nag)
        
    # read data to numpy.ndarray
    list_element_na = xda_list_element_na.values.astype(int)
    list_element_coords = xda_list_element_coords.values
    dict_list_element = {}
    dict_data_wave = {}
    for nag in na_grid:
        dict_list_element[nag] = dict_xda_list_element[nag].values.astype(int)
        if load_wave_data:
            dict_data_wave[nag] = dict_xda_data_wave[nag].values
        
    ############### return ################
    if load_wave_data:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_data_wave
    else:
        return na_grid, data_time, list_element_na, list_element_coords, dict_list_element, dict_xda_data_wave

# Inplane slices

## Read

In [6]:
# data dir
# ModelNameList = ['model0021']
ModelNameList = []
for imodel in [3000]:
    ModeName = 'LatinSphericalHarmonicsAcousticBall%04d' %imodel
    ModelNameList.append(ModeName)
    
# wave dimension to animation
output_channel = 'X'
wave_dim_X = output_channel.index('X')

# output_channel = 'Xspz'
# wave_dim_s = output_channel.index('s')
# wave_dim_p = output_channel.index('p')
# wave_dim_z = output_channel.index('z')
# wave_dim_X = output_channel.index('X')

for ModelName in ModelNameList:
    data_dir = '../Runs/%s/output/elements/full_coeff' %ModelName
    
    if not os.path.exists(data_dir):
        print('ERROR %s not found' %data_dir)
        continue
    
    # read
    na_grid, data_time, list_element_na, list_element_coords, \
    dict_list_element, dict_data_wave = read_element_output(data_dir)

        # print(ModelName, "reading error!!!!!")
        # continue

    # time steps
    ntime = len(data_time)

    # # phi of the slices
    nag = na_grid[0]

    # GLL coords on elements
    nelem = list_element_coords.shape[0]
    ngll = list_element_coords.shape[1]
    # flattened coords, (s, z)
    element_coords_sz = list_element_coords.reshape((nelem * ngll), 2)

    # # save element_coords_sz
    # element_coords_sz

    wave_X = np.ndarray((nag, nelem * ngll))

    # for itime in np.arange(ntime):
    itime = 10

    for ielem in np.arange(nelem):
        wave_X[:, (ielem * ngll):(ielem * ngll + ngll)] = dict_data_wave[nag][ielem, :, :, wave_dim_X, itime]


    #     # make slice for phi
    #     NETCDFDir = data_dir + '/netcdf_coeff'
    #     os.makedirs(NETCDFDir, exist_ok=True)

    #     nc = Dataset(NETCDFDir+'/phi_coef_time%d.nc' %itime, 'w')
    #     nc.createDimension('npoint', size=nelem * ngll)
    #     nc.createDimension('Nr_Dim', size=nag)
    #     nc.createDimension('d2', size=2)
    #     # nc.createDimension('ntime', size=ntime)

    #     nc.createVariable('element_coords_sz', float, dimensions=('npoint','d2'))
    #     nc['element_coords_sz'][:] = element_coords_sz[:]
    #     # nc.createVariable('time', float, dimensions=('ntime'))
    #     # nc['time'][:] = data_time[:]

    #     nc.createVariable('X_coef', float, dimensions=('Nr_Dim', 'npoint'))
    #     nc['X_coef'][:] = wave_X[:]

    #     nc.close()

    # # Check Repeated Dataset
    # if os.path.exists('../DataSet/%s/' %ModelName):
    #     shutil.rmtree('../DataSet/%s/' %ModelName)

    # Initial Model Folder
    # os.makedirs('../DataSet/%s/' %ModelName, exist_ok=True)

    # # Make time info file
    # if os.path.exists('../DataSet/%s/time_info.nc' %ModelName):
    #     os.remove('../DataSet/%s/time_info.nc' %ModelName)
    # nc = Dataset('../DataSet/%s/time_info.nc' %ModelName, 'w')
    # nc.createDimension('ntime', size=ntime)
    # nc.createVariable('time', float, dimensions=('ntime'))
    # nc['time'][:] = data_time[:]

    # # Check 3D model nc file
    # NCFilePath = glob.glob('../Runs/%s/input/*.nc' %ModelName)
    # if len(NCFilePath) > 0:
    #     for path in NCFilePath:
    #         shutil.copy(path, '../DataSet/%s/' %ModelName)
    # # Move Spherical Harmonics Paramters
    # SphericalHarmonicsPath = glob.glob('../Runs/%s/Spherical_Harmonics.pkl' %ModelName)[0]
    # shutil.copy(SphericalHarmonicsPath, '../DataSet/%s/' %ModelName)

    # target_dir = '../DataSet/%s/snapshot_coeff/' %ModelName
    # if os.path.exists(target_dir):
    #     shutil.rmtree(target_dir)
    # shutil.move(data_dir + '/netcdf_coeff/', target_dir)

    # print(data_dir + '/netcdf_coeff/', target_dir)

In [4]:
data_time

array([0.0748784, 0.2747896, 0.4747008, 0.674612 , 0.8745232, 1.0744344,
       1.2743456, 1.4742568, 1.674168 , 1.8740792, 2.0739904, 2.2739016,
       2.4738128, 2.673724 , 2.8736352])

In [45]:
phi = 0.39269908
itime = 10
# vtk mesh
xyz = np.ndarray((nelem * ngll, 3))
xyz[:, 0] = element_coords_sz[:, 0] * np.cos(phi)
xyz[:, 1] = element_coords_sz[:, 0] * np.sin(phi)
xyz[:, 2] = element_coords_sz[:, 1]

np.shape(element_coords_sz), np.shape(xyz), np.shape(wave_X)

i_test_element = 100
element_coords_sz[i_test_element,:], xyz[i_test_element,:], wave_X[:, i_test_element]

print('phi: ', phi)
print('element_coords_sz: ', element_coords_sz[i_test_element,:])
print('xyz : ', xyz[i_test_element,:])
print('wave_X coeff: ', wave_X[:, i_test_element])

phi:  0.39269908
element_coords_sz:  [378.13297608 266.31128853]
xyz :  [349.34931742 144.70522458 266.31128853]
wave_X coeff:  [ 2.77692624e-08 -1.22475186e-08  1.53417705e-08 -4.35420011e-09
  2.00585539e-08  1.37557068e-08  2.76780696e-08 -1.48278145e-08
  2.52795923e-10 -7.00466440e-09 -1.47456936e-09  9.11505704e-10
 -1.19077115e-09 -1.29248168e-09 -1.34939615e-10 -1.57163671e-09]


In [37]:
NrCoeff = wave_X[:, i_test_element]

Sum = 0
for Nr in range(0, len(NrCoeff)):
    # print(Nr)
    Sum += NrCoeff[Nr]*np.exp(1j*Nr*phi)

print(Sum)

(1.9370202484033804e-08+5.476560293853236e-08j)


In [46]:
NrCoeff = wave_X[:, i_test_element]

Sum = 0
for iNr, Nr in enumerate(range(-len(NrCoeff)//2, len(NrCoeff)//2)):
    Sum += NrCoeff[iNr]*np.exp(1j*Nr*phi)

print(Sum)

(-1.937020173978057e-08-5.4765603201769394e-08j)


In [47]:
abs(Sum)

5.8090240225819794e-08

array([[[[[ 0.00000000e+00,  0.00000000e+00, -2.73118312e-22, ...,
            1.59857379e-07,  3.46200508e-08,  6.80571873e-07]]],


        [[[ 0.00000000e+00,  0.00000000e+00,  4.19138031e-22, ...,
            4.29899814e-08, -1.02467105e-07,  1.54086450e-07]]],


        [[[ 0.00000000e+00,  0.00000000e+00,  1.77777037e-22, ...,
           -3.19699076e-08,  1.43854592e-07, -6.02664159e-08]]],


        ...,


        [[[ 0.00000000e+00,  0.00000000e+00, -2.18974897e-23, ...,
           -1.70267012e-09,  6.09568929e-09,  2.78792900e-08]]],


        [[[ 0.00000000e+00,  0.00000000e+00,  2.91834169e-23, ...,
            2.28356112e-09,  1.69098235e-09,  5.43451417e-09]]],


        [[[ 0.00000000e+00,  0.00000000e+00,  4.86190595e-24, ...,
            2.61128763e-09, -8.72710793e-09, -1.30492781e-08]]]],



       [[[[ 0.00000000e+00,  0.00000000e+00, -2.58345115e-23, ...,
            1.58020086e-07,  3.07725578e-08,  1.51880181e-06]]],


        [[[ 0.00000000e+00,  0.00000000e+00, 